In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, PReLU, Input, Embedding, Lambda
from keras.optimizers import Adam
import NnVisualizer

from keras.datasets import mnist
from keras.utils import np_utils

# Traditional Softmax loss with visualization

In [ ]:
# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# data pre-processing
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],  1) / 255.   # normalize
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1) / 255.      # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)    #one hot
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [ ]:
# Create model
nn_input = Input(shape=(28, 28, 1))
nn = Conv2D(32, (3, 3))(nn_input)
nn = PReLU()(nn)
nn = Conv2D(32, (3, 3))(nn)
nn = PReLU()(nn)
nn = MaxPooling2D()(nn)
nn = Conv2D(64, (3, 3))(nn)
nn = PReLU()(nn)
nn = Conv2D(64, (3, 3))(nn)
nn = PReLU()(nn)
nn = MaxPooling2D()(nn)
nn = Flatten()(nn)
nn = Dense(2)(nn)
nn = PReLU(name = 'visualize_layer')(nn)
nn = Dense(10, activation = 'softmax')(nn)
model = Model(inputs = nn_input, outputs = nn)

# We add metrics to get more results you want to see
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print('Training ------------')
visualize_callback = NnVisualizer.Histories()
model.fit(x_train, y_train, epochs=2, batch_size=32, validation_data=([x_test, y_test]), callbacks=[visualize_callback])

In [ ]:
print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(x_test, y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)

# Center loss with visualization

In [ ]:
(x_train, y_train_label), (x_test, y_test_label) = mnist.load_data()

# data pre-processing
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],  1) / 255.   # normalize
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1) / 255.      # normalize
y_train = np_utils.to_categorical(y_train_label, num_classes=10)    #one hot
y_test = np_utils.to_categorical(y_test_label, num_classes=10)    #one hot

In [ ]:
# Create softmax model
nn_input = Input(shape=(28, 28, 1))
nn = Conv2D(32, (3, 3))(nn_input)
nn = PReLU()(nn)
nn = Conv2D(32, (3, 3))(nn)
nn = PReLU()(nn)
nn = MaxPooling2D()(nn)
nn = Conv2D(64, (3, 3))(nn)
nn = PReLU()(nn)
nn = Conv2D(64, (3, 3))(nn)
nn = PReLU()(nn)
nn = MaxPooling2D()(nn)
nn = Flatten()(nn)
nn = Dense(2)(nn)
visualize_feature = PReLU(name = 'visualize_layer')(nn)
nn = Dense(10, activation = 'softmax')(visualize_feature)
model = Model(inputs = nn_input, outputs = nn)


model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Define dum of square error lambda
l2 = lambda x: np.sum((x[0] - x[1])**2, keepdims = True)

# Create prior model (center loss)
prior_nn_input = Input(shape=(1,))   # single value ground truth labels as inputs
centers = Embedding(10, 2)(prior_nn_input)  # Computer center
centers = Flatten()(centers)  # visualize_feature.shape = (?, 2) centers.shape = (?, 1, 2)
lc = Lambda(l2, name = 'lc')([visualize_feature, centers])
prior_model = Model(inputs=[nn_input, prior_nn_input], outputs=[nn, lc])

prior_model.compile(optimizer=Adam(), 
                    loss=["categorical_crossentropy", lambda true, pred: pred],
                    loss_weights=[1, 0.2], metrics=['accuracy'])

In [ ]:
import NnVisualizer
print('Training ------------')
visualize_callback = NnVisualizer.Histories(True)
random_y_train = np.random.rand(x_train.shape[0], 1)
random_y_test = np.random.rand(x_test.shape[0], 1)

prior_model.fit([x_train, y_train_label], [y_train, random_y_train], epochs=2, batch_size=32, validation_data=([x_test, y_test_label], [y_test, random_y_test]), callbacks=[visualize_callback])

In [ ]:
print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(x_test, y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)